In [1]:
%matplotlib inline
import os

import intake
import satsearch
from satsearch import Search

import ast
import pandas as pd
import numpy as np
import xarray as xr

import geopandas as gpd
import rasterio
import rioxarray

import dask
from dask.diagnostics import ProgressBar

In [2]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='250000000',
           GDAL_SWATH_SIZE='250000000',
           VSI_CURL_CACHE_SIZE='250000000')
os.environ.update(env)


In [3]:
bbox = [35.48, -3.24, 35.58, -3.14] # (min lon, min lat, max lon, max lat)
bbox = [129.0000000000000000,-39.0000000000000000, 141.0000000000000000,-26.0000000000000000]
#bbox = [138.54207373854032 - 0.01, -34.934374426311145 - 0.01, 138.54207373854032 + 0.01, -34.934374426311145 + 0.01]
#bbox = [138.54207373854032 - 0.5, -34.934374426311145 - 0.5, 138.54207373854032 + 0.5, -34.934374426311145 + 0.5]

dates = '2017-01-01/2020-12-31'
dates = '2020-01-01/2020-12-31'

URL='https://earth-search.aws.element84.com/v0'
results = satsearch.Search.search(url=URL,
                                  collections=['sentinel-s2-l2a-cogs'], # note collection='sentinel-s2-l2a-cogs' doesn't work
                                  datetime=dates,
                                  bbox=bbox,   
                                  #property=["eo:cloud_cover<1"],
                                  #query={'eo:cloud_cover': {'lt': 1}},
                                  query={'eo:cloud_cover': {'eq': 0}, 'platform':{'eq': 'sentinel-2a'}},
                                  sort=['<datetime'])



In [4]:
print('%s items' % results.found())
items = results.items()
# Save this locally for use later
#items.save('my-s2-l2a-cogs-sa-cloudcover1.json')
items.save('my-s2-l2a-cogs-sa-cloudcover0OZLocation.geojson')


catalog = intake.open_stac_item_collection(results.items())


2726 items


In [5]:
gf2 = gpd.read_file('my-s2-l2a-cogs-sa-cloudcover0OZLocation.geojson')

gf2.head()


,id,datetime,platform,constellation,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated,geometry
0,S2A_54JVQ_20201230_0_L2A,2020-12-30T00:54:41+00:00,sentinel-2a,sentinel-2,10,0,32754,54,J,VQ,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JVQ_2...,79.40,0,True,2020-12-30T06:47:24.098000+00:00,2020-12-30T06:47:24.098000+00:00,"POLYGON ((140.74666 -28.11348, 139.98154 -28.1..."
1,S2A_54JUR_20201230_0_L2A,2020-12-30T00:54:30+00:00,sentinel-2a,sentinel-2,10,0,32754,54,J,UR,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JUR_2...,100.00,0,True,2020-12-30T06:48:49.867001+00:00,2020-12-30T06:48:49.867001+00:00,"POLYGON ((140.08920 -27.20780, 138.98092 -27.1..."
2,S2A_53JRM_20201230_0_L2A,2020-12-30T00:54:19+00:00,sentinel-2a,sentinel-2,10,0,32753,53,J,RM,0,S2A_MSIL2A_20201230T004711_N0214_R102_T53JRM_2...,72.54,0,True,2020-12-30T06:50:26.423000+00:00,2020-12-30T06:50:26.423000+00:00,"POLYGON ((138.15656 -26.27295, 138.42514 -25.2..."
3,S2A_54JTS_20201230_0_L2A,2020-12-30T00:54:19+00:00,sentinel-2a,sentinel-2,10,0,32754,54,J,TS,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JTS_2...,74.42,0,True,2020-12-30T06:52:50.251999+00:00,2020-12-30T06:52:50.251999+00:00,"POLYGON ((139.09470 -26.29503, 138.15479 -26.2..."
4,S2A_54JUS_20201230_0_L2A,2020-12-30T00:54:16+00:00,sentinel-2a,sentinel-2,10,0,32754,54,J,US,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JUS_2...,100.00,0,True,2020-12-30T07:30:12.892000+00:00,2020-12-30T07:30:12.892000+00:00,"POLYGON ((140.09637 -26.30484, 138.99680 -26.2..."


In [6]:
def get_STAC_items(url, collection, dates, bbox):
    results = Search.search(url=url,
                        collections=[collection], 
                        datetime=dates,
                        bbox=bbox,    
                        query={'eo:cloud_cover': {'eq': 0}, 'platform':{'eq': 'sentinel-2a'}},
                        sortby=['-properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} Items')
    
    return intake.open_stac_item_collection(items)


In [7]:
URL2='https://earth-search.aws.element84.com/v0'
collection2 = 'sentinel-s2-l2a-cogs' # sentinel-2 w/ atmospheric corrections
from satsearch import Search
items_s2 = get_STAC_items(URL2, collection2, dates, bbox)


Found 10000 Items


In [8]:
gf2function = items_s2.to_geopandas()
gf2function.head()


/home/ec2-user/anaconda3/envs/daskpy3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry,datetime,platform,constellation,instruments,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated
0,"POLYGON ((140.74666 -28.11348, 139.98154 -28.1...",2020-12-30T00:54:41Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,J,VQ,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JVQ_2...,79.40,0,True,2020-12-30T06:47:24.098Z,2020-12-30T06:47:24.098Z
1,"POLYGON ((140.08920 -27.20780, 138.98092 -27.1...",2020-12-30T00:54:30Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,J,UR,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JUR_2...,100.00,0,True,2020-12-30T06:48:49.867Z,2020-12-30T06:48:49.867Z
2,"POLYGON ((138.15656 -26.27295, 138.42514 -25.2...",2020-12-30T00:54:19Z,sentinel-2a,sentinel-2,[msi],10,0,32753,53,J,RM,0,S2A_MSIL2A_20201230T004711_N0214_R102_T53JRM_2...,72.54,0,True,2020-12-30T06:50:26.423Z,2020-12-30T06:50:26.423Z
3,"POLYGON ((139.09470 -26.29503, 138.15479 -26.2...",2020-12-30T00:54:19Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,J,TS,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JTS_2...,74.42,0,True,2020-12-30T06:52:50.252Z,2020-12-30T06:52:50.252Z
4,"POLYGON ((140.09637 -26.30484, 138.99680 -26.2...",2020-12-30T00:54:16Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,J,US,0,S2A_MSIL2A_20201230T004711_N0214_R102_T54JUS_2...,100.00,0,True,2020-12-30T07:30:12.892Z,2020-12-30T07:30:12.892Z


In [9]:
import boto3
import botocore
import datetime

ecs = boto3.client('ecs')
resp = ecs.list_clusters()
clusters = resp['clusterArns']
if len(clusters) > 1:
    print("Please manually select your cluster")
cluster = clusters[0]
cluster

'arn:aws:ecs:us-west-2:867753693528:cluster/era5-Fargate-Dask-Cluster'

In [10]:
import matplotlib.pyplot as plt
import matplotlib
import s3fs
import fsspec
from dask.distributed import performance_report
from dask.distributed import Client

numWorkers=70
numWorkers=100
numWorkers=150
numWorkers=300
ecs.update_service(cluster=cluster, service='Dask-Worker', desiredCount=numWorkers)
ecs.get_waiter('services_stable').wait(cluster=cluster, services=['Dask-Worker'])

In [11]:
client = Client('Dask-Scheduler:8786')
#client = Client('Dask-Scheduler.local-dask:8786')
client

OSError: Timed out trying to connect to tcp://Dask-Scheduler:8786 after 10 s

In [12]:
%%time

@dask.delayed
def stacitem_to_dataset2(item, datetimeuse):
    print(item)
    #stacked = catalog[item.id].stack_bands(bands)
    #stacked = item.stack_bands(['B02','B03','B04','B05'])
    stacked = item.stack_bands(['B02'])
    da = stacked(chunks=dict(band=1, x=2048, y=2048)).to_dask()
    #da = stacked(chunks=dict(band=1, x=5120, y=5120)).to_dask()
    #da['band'] = bands # use common names
    #da['band'] = ['blue','green','red','nir'] # use common names
    da['band'] = ['blue'] # use common names
    da = da.expand_dims(time=[pd.to_datetime(datetimeuse)])
    #da = stacked(chunks=dict(band=1, x=5120, y=5120, time=-1)).to_dask()
    ds = da.to_dataset(dim='band')
    return ds

def stacitem_to_dataset3(item, datetimeuse, utmzone):
    #print(item)
    #stacked = catalog[item.id].stack_bands(bands)
    #stacked = item.stack_bands(['B02','B03','B04','B05'])
    stacked = item.stack_bands(['B02'])
    da = stacked(chunks=dict(band=1, x=2048, y=2048)).to_dask()
    #da = stacked(chunks=dict(band=1, x=5120, y=5120)).to_dask()
    #da['band'] = bands # use common names
    #da['band'] = ['blue','green','red','nir'] # use common names
    da['band'] = ['blue'] # use common names
    da = da.expand_dims(time=[pd.to_datetime(datetimeuse)])
    #da = stacked(chunks=dict(band=1, x=5120, y=5120, time=-1)).to_dask()
    ds = da.to_dataset(dim='band')
    return ds

lazy_datasets = []
#for i,item in gf.iterrows():
#for i,item in gf.head(1).iterrows():   

for index, x in enumerate(items_s2):
    #need to make items_s2
    #need to get datetime from the dataframe gf2 as well
    utmzone = gf2function.iloc[index]['sentinel:utm_zone']
    datetimeuse = gf2function.iloc[index]['datetime']

    itemx = items_s2[x]
    ds = stacitem_to_dataset3(itemx, datetimeuse, utmzone)
    if utmzone == 52:
    #if utmzone == 53:
    #if utmzone == 54:        
        lazy_datasets.append(ds)
        #break

print(len(lazy_datasets))        

116
CPU times: user 17.6 s, sys: 189 ms, total: 17.8 s
Wall time: 1min 10s


In [ ]:
#SKIP FOR NOW

from dask.diagnostics import ProgressBar
with ProgressBar():
    #out = dask.compute(*lazy_datasets)
    
#datasets = dask.compute(*lazy_datasets).mean(dim='time')
    datasets = dask.compute(*lazy_datasets)


In [13]:
DS = xr.concat(lazy_datasets, dim='time')
DS


/home/ec2-user/anaconda3/envs/daskpy3/lib/python3.7/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/home/ec2-user/anaconda3/envs/daskpy3/lib/python3.7/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[in

,Array,Chunk
Bytes,4.78 TB,33.55 MB
Shape,"(116, 100980, 50982)","(1, 2048, 2048)"
Count,491311 Tasks,153816 Chunks
Type,float64,numpy.ndarray


In [14]:
DS['blue']

,Array,Chunk
Bytes,4.78 TB,33.55 MB
Shape,"(116, 100980, 50982)","(1, 2048, 2048)"
Count,491311 Tasks,153816 Chunks
Type,float64,numpy.ndarray


In [15]:
def median(array, dim, keep_attrs=False, skipna=False, **kwargs):
    """ Runs a median on an dask-backed xarray.
    
    This function does not scale!
    It will rechunk along the given dimension, so make sure 
    your other chunk sizes are small enough that it 
    will fit into memory.
    
    :param DataArray array: An xarray.DataArray wrapping a dask array
    :param dim str: The name of the dim in array to calculate the median
    """
    if type(array) is xr.Dataset:
        return array.apply(median, dim=dim, keep_attrs=keep_attrs, **kwargs)
    
    if not hasattr(array.data, 'dask'):
        return array.median(dim, keep_attrs=keep_attrs, **kwargs)
    
    array = array.chunk({dim:-1})
    axis = array.dims.index(dim)
    median_func = np.nanmedian if skipna else np.median
    blocks = dask.array.map_blocks(median_func, array.data, dtype=array.dtype, drop_axis=axis, axis=axis, **kwargs)
    
    new_coords={k: v for k, v in array.coords.items() if k != dim and dim not in v.dims}
    new_dims = tuple(d for d in array.dims if d != dim)
    new_attrs = array.attrs if keep_attrs else None
    
    return xr.DataArray(blocks, coords=new_coords, dims=new_dims, attrs=new_attrs)

In [16]:
blue_median = median(DS.blue, dim='time')

In [17]:
blue_median

<xarray.DataArray 'median-2193710c1ac594c27e7d7aabc93b2a78' (y: 100980, x: 50982)>
dask.array<median, shape=(100980, 50982), dtype=float64, chunksize=(2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 4e+05 4e+05 4e+05 ... 9.098e+05 9.098e+05 9.098e+05
  * y        (y) float64 6.19e+06 6.19e+06 6.19e+06 ... 7.2e+06 7.2e+06 7.2e+06

In [18]:
with ProgressBar():
    bluemedianNP = blue_median.compute()


[                                        ] | 0% Completed | 18.8s


MemoryError: Unable to allocate 2.68 GiB for an array with shape (1, 70736, 40742) and data type bool

In [ ]:
env